In [3]:
import pandas as pd

def process(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df['month'] = df['timestamp'].dt.month
    df['year'] = df['timestamp'].dt.year


    df['new_date'] = df.apply(lambda row: row['timestamp'] + pd.DateOffset(days=row['billing_period']), axis=1)
    df['month_new'] = df['new_date'].dt.month
    df['year_new'] = df['new_date'].dt.year

    # Функция для подсчёта количества дней в феврале
    def count_days_in_february(date_start, date_end):
        february_start = pd.Timestamp(date_start.year, 2, 1)
        february_end = pd.Timestamp(date_start.year, 3, 1) - pd.DateOffset(days=1)
        overlap_start = max(february_start, date_start)
        overlap_end = min(february_end, date_end)
        overlap_days = max((overlap_end - overlap_start).days + 1, 0)
        return overlap_days

    # Проверка, содержатся ли даты в феврале 2024 года и подсчёт количества дней в феврале
    df['days_in_february'] = df.apply(lambda x: count_days_in_february(x['timestamp'], x['new_date']), axis=1)

    df.loc[((df['month'] == 1) & (df['month_new'] == 2)) | ((df['year'] < 2024) & (df['month_new'] == 2)), 'days_in_february'] -=1

    df['cost'] = df['billing_total_price_usd'] / df['billing_period'] * df['days_in_february']
    df = df.groupby('user_id', as_index=False).agg({'cost': 'sum'})
    res = (df['cost'].nlargest(3).sum())
    res = round(res, 2)
    return res

In [4]:
df = pd.read_csv("sample_2.csv")
print(process(df))

30.0


In [1]:
import pandas as pd
df = pd.read_csv("sample_1.csv")
df

,id,user_id,timestamp,billing_period,billing_total_price_usd
0,0,26,1708104067,74,95
1,1,40,1707585667,25,46
2,2,48,1704993667,50,99
3,3,15,1704475267,32,15
4,4,38,1706030467,84,83


In [2]:
df.at[3, 'user_id'] = 40
df

,id,user_id,timestamp,billing_period,billing_total_price_usd
0,0,26,1708104067,74,95
1,1,40,1707585667,25,46
2,2,48,1704993667,50,99
3,3,40,1704475267,32,15
4,4,38,1706030467,84,83


In [3]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df['month'] = df['timestamp'].dt.month
df['year'] = df['timestamp'].dt.year


df['new_date'] = df.apply(lambda row: row['timestamp'] + pd.DateOffset(days=row['billing_period']), axis=1)
df['month_new'] = df['new_date'].dt.month
df['year_new'] = df['new_date'].dt.year

df

,id,user_id,timestamp,billing_period,billing_total_price_usd,month,year,new_date,month_new,year_new
0,0,26,2024-02-16 17:21:07,74,95,2,2024,2024-04-30 17:21:07,4,2024
1,1,40,2024-02-10 17:21:07,25,46,2,2024,2024-03-06 17:21:07,3,2024
2,2,48,2024-01-11 17:21:07,50,99,1,2024,2024-03-01 17:21:07,3,2024
3,3,40,2024-01-05 17:21:07,32,15,1,2024,2024-02-06 17:21:07,2,2024
4,4,38,2024-01-23 17:21:07,84,83,1,2024,2024-04-16 17:21:07,4,2024


In [4]:
# Функция для подсчёта количества дней в феврале
def count_days_in_february(date_start, date_end):
    february_start = pd.Timestamp(date_start.year, 2, 1)
    february_end = pd.Timestamp(date_start.year, 3, 1) - pd.DateOffset(days=1)
    overlap_start = max(february_start, date_start)
    overlap_end = min(february_end, date_end)
    overlap_days = max((overlap_end - overlap_start).days + 1, 0)
    return overlap_days

# Проверка, содержатся ли даты в феврале 2024 года и подсчёт количества дней в феврале
df['days_in_february'] = df.apply(lambda x: count_days_in_february(x['timestamp'], x['new_date']), axis=1)

df

,id,user_id,timestamp,billing_period,billing_total_price_usd,month,year,new_date,month_new,year_new,days_in_february
0,0,26,2024-02-16 17:21:07,74,95,2,2024,2024-04-30 17:21:07,4,2024,13
1,1,40,2024-02-10 17:21:07,25,46,2,2024,2024-03-06 17:21:07,3,2024,19
2,2,48,2024-01-11 17:21:07,50,99,1,2024,2024-03-01 17:21:07,3,2024,29
3,3,40,2024-01-05 17:21:07,32,15,1,2024,2024-02-06 17:21:07,2,2024,6
4,4,38,2024-01-23 17:21:07,84,83,1,2024,2024-04-16 17:21:07,4,2024,29


In [5]:
df.loc[((df['month'] == 1) & (df['month_new'] == 2)) | ((df['year'] < 2024) & (df['month_new'] == 2)), 'days_in_february'] -=1
df['cost'] = df['billing_total_price_usd'] / df['billing_period'] * df['days_in_february']
df


,id,user_id,timestamp,billing_period,billing_total_price_usd,month,year,new_date,month_new,year_new,days_in_february,cost
0,0,26,2024-02-16 17:21:07,74,95,2,2024,2024-04-30 17:21:07,4,2024,13,16.689189
1,1,40,2024-02-10 17:21:07,25,46,2,2024,2024-03-06 17:21:07,3,2024,19,34.960000
2,2,48,2024-01-11 17:21:07,50,99,1,2024,2024-03-01 17:21:07,3,2024,29,57.420000
3,3,40,2024-01-05 17:21:07,32,15,1,2024,2024-02-06 17:21:07,2,2024,5,2.343750
4,4,38,2024-01-23 17:21:07,84,83,1,2024,2024-04-16 17:21:07,4,2024,29,28.654762


In [6]:
df = df.groupby('user_id', as_index=False).agg({'cost': 'sum'})
df

,user_id,cost
0,26,16.689189
1,38,28.654762
2,40,37.303750
3,48,57.420000


In [7]:

res = (df['cost'].nlargest(3).sum())
res

123.3785119047619

In [12]:
process(df)

123.38